In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
stream = r'/home/jupyter/datasphere/project/65/model.bin'
wv = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv(r"/home/jupyter/datasphere/project/CW_dataset/CW_dataset.csv")

In [4]:
data.head()

,text,label
0,№107 29 октября 1919 г. Председательствовал: г...,5
1,ОБЪЯВЛЕНИЕ ГЛАВНОГО СОВЕТА СРН О ВОЗМОЖНОСТИ ...,4
2,№13 14 ноября 1918 г. Присутствовали: генерал ...,5
3,1 октября 1920 г. I. Работа южной группы «Союз...,5
4,"человека к работе над собою неустанной, что ос...",7


In [9]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [5]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [6]:
nlp = spacy.load("ru_core_news_sm")
stop_words = nlp.Defaults.stop_words


In [7]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
data['tokens'] = data['text'].apply(spacy_tokenizer)

In [16]:
data.head()

,text,label,tokens,vec
0,№107 29 октября 1919 г. Председательствовал: г...,5,"[№, 107, 29, октябрь, 1919, г., председательст...","[0.02356454943732923, -0.25588998527375245, 0...."
1,ОБЪЯВЛЕНИЕ ГЛАВНОГО СОВЕТА СРН О ВОЗМОЖНОСТИ ...,4,"[объявление, главного, совета, срн, возможност...","[0.009499084349405372, -0.23016576355165047, 0..."
2,№13 14 ноября 1918 г. Присутствовали: генерал ...,5,"[№, 13, 14, ноябрь, 1918, г., присутствовать, ...","[-0.04520371645941993, -0.257928786047805, 0.0..."
3,1 октября 1920 г. I. Работа южной группы «Союз...,5,"[1, октябрь, 1920, г., i., работа, южный, груп...","[0.025100602099751877, -0.21974555051430264, 0..."
4,"человека к работе над собою неустанной, что ос...",7,"[человек, работа, неустанный, освобождение, ду...","[-0.10345465863907807, -0.3687248403500569, -0..."


In [15]:
data['vec'] = data['tokens'].apply(sent_vec)

In [17]:
data.head()

,text,label,tokens,vec
0,№107 29 октября 1919 г. Председательствовал: г...,5,"[№, 107, 29, октябрь, 1919, г., председательст...","[0.02356454943732923, -0.25588998527375245, 0...."
1,ОБЪЯВЛЕНИЕ ГЛАВНОГО СОВЕТА СРН О ВОЗМОЖНОСТИ ...,4,"[объявление, главного, совета, срн, возможност...","[0.009499084349405372, -0.23016576355165047, 0..."
2,№13 14 ноября 1918 г. Присутствовали: генерал ...,5,"[№, 13, 14, ноябрь, 1918, г., присутствовать, ...","[-0.04520371645941993, -0.257928786047805, 0.0..."
3,1 октября 1920 г. I. Работа южной группы «Союз...,5,"[1, октябрь, 1920, г., i., работа, южный, груп...","[0.025100602099751877, -0.21974555051430264, 0..."
4,"человека к работе над собою неустанной, что ос...",7,"[человек, работа, неустанный, освобождение, ду...","[-0.10345465863907807, -0.3687248403500569, -0..."


In [18]:
X = data['vec'].to_list()
y = data['label'].to_list()

In [19]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, shuffle=True)

## LogisticRegression

In [2]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=3000)

In [ ]:
classifier.fit(X_train,y_train)

In [20]:
from sklearn import metrics
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression F1:",metrics.precision_score(y_test, predicted, average='macro'))


Logistic Regression Accuracy: 0.7793814432989691
Logistic Regression F1: 0.7974874812459888


In [21]:
from sklearn.metrics import classification_report


target_names = ['Bolsheviki', 'Mensheviki', 'Left_SRs', 'Kadets', 'Rights', 'White_army', 'Work_oppose', 'Nobles']

print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

  Bolsheviki       0.88      0.69      0.77        52
  Mensheviki       0.62      0.73      0.67        62
    Left_SRs       0.79      0.77      0.78        70
      Kadets       0.88      0.78      0.83        37
      Rights       0.88      0.76      0.81        58
  White_army       0.86      0.86      0.86        56
 Work_oppose       0.73      0.57      0.64        42
      Nobles       0.75      0.91      0.82       108

    accuracy                           0.78       485
   macro avg       0.80      0.76      0.77       485
weighted avg       0.79      0.78      0.78       485



## RandomForest

In [17]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [18]:
from sklearn import metrics
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression F1:",metrics.precision_score(y_test, predicted, average='macro'))

Logistic Regression Accuracy: 0.8061855670103093
Logistic Regression F1: 0.8357587432899796


In [19]:
from sklearn.metrics import classification_report
target_names = ['Bolsheviki', 'Mensheviki', 'Left_SRs', 'Kadets', 'Rights', 'White_army', 'Work_oppose', 'Nobles']

print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

  Bolsheviki       1.00      0.77      0.87        52
  Mensheviki       0.62      0.77      0.69        62
    Left_SRs       0.81      0.79      0.80        70
      Kadets       0.93      0.70      0.80        37
      Rights       0.90      0.76      0.82        58
  White_army       0.82      0.84      0.83        56
 Work_oppose       0.82      0.79      0.80        42
      Nobles       0.78      0.91      0.84       108

    accuracy                           0.81       485
   macro avg       0.84      0.79      0.81       485
weighted avg       0.82      0.81      0.81       485



## XGBoost

In [ ]:
import xgboost as xgb

# Instantiate classifier with mostly default hyperparameters
xgb_classifier = xgb.XGBClassifier(n_estimators=1000, reg_lambda=1, gamma=0, max_depth=9, objective="multi:softmax",
                                   verbosity=3)
# Fit classifier
xgb_classifier.fit(X_train, y_train)

In [ ]:
from sklearn import metrics
predicted = xgb_classifier.predict(X_test)

In [22]:
from sklearn.metrics import classification_report
target_names = ['Bolsheviki', 'Mensheviki', 'Left_SRs', 'Kadets', 'Rights', 'White_army', 'Work_oppose', 'Nobles']

print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

  Bolsheviki       0.87      0.77      0.82        52
  Mensheviki       0.70      0.77      0.73        62
    Left_SRs       0.85      0.76      0.80        70
      Kadets       0.88      0.81      0.85        37
      Rights       0.79      0.79      0.79        58
  White_army       0.82      0.91      0.86        56
 Work_oppose       0.85      0.79      0.81        42
      Nobles       0.85      0.91      0.88       108

    accuracy                           0.82       485
   macro avg       0.83      0.81      0.82       485
weighted avg       0.83      0.82      0.82       485



## MultinomialNB 

In [10]:
data['tokens'] = data['text'].apply(spacy_tokenizer)

In [17]:
data.head()

,text,label,tokens,joined_texts
0,№107 29 октября 1919 г. Председательствовал: г...,5,"[№, 107, 29, октябрь, 1919, г., председательст...",№ 107 29 октябрь 1919 г. председательствовал г...
1,ОБЪЯВЛЕНИЕ ГЛАВНОГО СОВЕТА СРН О ВОЗМОЖНОСТИ ...,4,"[объявление, главного, совета, срн, возможност...",объявление главного совета срн возможность при...
2,№13 14 ноября 1918 г. Присутствовали: генерал ...,5,"[№, 13, 14, ноябрь, 1918, г., присутствовать, ...",№ 13 14 ноябрь 1918 г. присутствовать генерал ...
3,1 октября 1920 г. I. Работа южной группы «Союз...,5,"[1, октябрь, 1920, г., i., работа, южный, груп...",1 октябрь 1920 г. i. работа южный группа союз ...
4,"человека к работе над собою неустанной, что ос...",7,"[человек, работа, неустанный, освобождение, ду...",человек работа неустанный освобождение духовны...


In [16]:
data['joined_texts'] = data['tokens'].apply(lambda x: ' '.join(x))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = data['joined_texts'].to_list()
y = data['label'].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, shuffle=True)



In [19]:
from nltk.corpus import stopwords as nltk_stopwords


vectorizer = TfidfVectorizer(stop_words='english')
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [20]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(X_train_vect, y_train)

MultinomialNB()

In [21]:
from sklearn.metrics import classification_report

y_pred = classifier.predict(X_test_vect)
target_names = ['Bolsheviki', 'Mensheviki', 'Left_SRs', 'Kadets', 'Rights', 'White_army', 'Work_oppose', 'Nobles']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

  Bolsheviki       1.00      0.52      0.68        52
  Mensheviki       0.63      0.95      0.76        62
    Left_SRs       0.95      0.76      0.84        70
      Kadets       1.00      0.65      0.79        37
      Rights       1.00      0.55      0.71        58
  White_army       0.93      0.75      0.83        56
 Work_oppose       1.00      0.26      0.42        42
      Nobles       0.53      0.97      0.69       108

    accuracy                           0.73       485
   macro avg       0.88      0.68      0.71       485
weighted avg       0.83      0.73      0.72       485

